In [1]:
from model import Model
import pickle
import numpy as np

with open('data/dimension.pickle', 'rb') as pk:
    dim_info = pickle.load(pk)
Sl_d, Sf_d, A_d = 1, dim_info['tages'], dim_info['tages']
T = 100

SCALE = 0.36

/home/shao.367/miniconda3/envs/pt-gpu/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
all_rewards = []

for user_class in range(10):
    model = Model(Sl_d, Sf_d, A_d, T, bias=0.3, normal_scale=SCALE)
    state_slow, state_fast = model.reset(user_class)
    rewards, actions, states_slow, states_fast = [], [], [], []
    A_prob = np.random.uniform(0, 1, size=len(state_fast))
    action_vec = np.zeros(len(A_prob))
    action = np.random.choice(len(A_prob), 1, p=A_prob/A_prob.sum()) 
    action_vec[action] = 1

    WINDOW_SIZE = 5

    while True:
        states_slow.append(state_slow)
        states_fast.append(state_fast)
        actions.append(action)

        state_slow, state_fast, terminal = model.update(state_slow, state_fast, action_vec, user_class)
        reward = model.reward(state_slow, state_fast, action_vec)
        
        if len(states_fast) >= WINDOW_SIZE:
            w_y = states_fast[:-WINDOW_SIZE] 
        else:
            w_y = states_fast
        y_current = np.array(w_y).mean(axis=0)


        action_vec = np.zeros(len(state_fast))
        if state_fast.sum() > 0:
            action = np.random.choice(len(action_vec), 1, p=state_fast/state_fast.sum())
        else:
            action = np.random.choice(len(action_vec), 1)
        action_vec[action] = 1
        rewards.append(reward)

        if terminal:
            all_rewards.append(np.array(rewards).sum())
            break
    
print(all_rewards)

with open(f'data/CB_rewards.pickle', 'wb') as pk:
    pickle.dump(all_rewards, pk, protocol=pickle.HIGHEST_PROTOCOL)

[2904.0760046376627, 2904.0759972004635, 2904.0876477176216, 2904.1599326471433, 2904.105476801149, 2904.0802437570183, 2904.1840263636254, 2904.0900433453767, 2904.313741831054, 2904.0884309906637]


/home/shao.367/miniconda3/envs/pt-gpu/lib/python3.9/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator KMeans from version 1.1.3 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
/tmp/ipykernel_137206/188901844.py:26: RuntimeWarning: Mean of empty slice.
  y_current = np.array(w_y).mean(axis=0)
/home/shao.367/miniconda3/envs/pt-gpu/lib/python3.9/site-packages/numpy/core/_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [28]:
heuristic_rewards = []

for user_class in range(10):
    model = Model(Sl_d, Sf_d, A_d, T, bias=0.3, normal_scale=SCALE)
    state_slow, state_fast = model.reset(user_class)
    rewards, actions, states_slow, states_fast = [], [], [], []
    A_prob = np.random.uniform(0, 1, size=len(state_fast))
    action_vec = np.zeros(len(A_prob))
    action = np.random.choice(len(A_prob), 1, p=A_prob/A_prob.sum()) 
    action_vec[action] = 1

    WINDOW_SIZE = 5
    UPDATE_WINDOWN = 10

    cnt = 0
    while True:
        
        states_slow.append(state_slow)
        states_fast.append(state_fast)
        actions.append(action)

        state_slow, state_fast, terminal = model.update(state_slow, state_fast, action_vec, user_class)
        reward = model.reward(state_slow, state_fast, action_vec)
        
        if len(states_fast) >= WINDOW_SIZE:
            w_y = states_fast[:-WINDOW_SIZE] 
        else:
            w_y = states_fast
        y_current = np.array(w_y).mean(axis=0)

        if cnt % UPDATE_WINDOWN == 0 and cnt != 0:
            action_vec = np.zeros(len(state_fast))
            if state_fast.sum() > 0:
                action = np.random.choice(len(action_vec), 1, p=state_fast/state_fast.sum())
            else:
                action = np.random.choice(len(action_vec), 1)
            action_vec[action] = 1
        rewards.append(reward)

        if terminal:
            heuristic_rewards.append(np.array(rewards).sum())
            break
        
        cnt += 1
    
print(heuristic_rewards)

[1509.689778645044, 514.9562258732418, 1311.5764850182343, 112.92995911615678, 1610.4885639620995, 2407.7521114567226, 1509.953136665449, 1013.2184049060061, 1810.958875061906, 1710.7905330480926]


/tmp/ipykernel_114552/2482508208.py:29: RuntimeWarning: Mean of empty slice.
  y_current = np.array(w_y).mean(axis=0)
